In [5]:
import h5py
import pickle
import numpy as np
import pylab as plt
from xfab.symmetry import Umis
from multiprocessing import Pool

In [6]:
%matplotlib inline
from scipy.ndimage import center_of_mass
from diffpy.structure import Atom, Lattice, Structure
import matplotlib.pyplot as plt
import numpy as np
from orix import plot, sampling
from orix.quaternion import Orientation, symmetry
from orix.crystal_map import Phase
from orix.quaternion import Rotation
from orix.vector import Miller
from orix.vector import Vector3d
from orix.quaternion import Misorientation
plt.rcParams.update(
    {
        "figure.figsize": (10, 5),
        "font.size": 20,
        "axes.grid": True,
        "lines.markersize": 10,
        "lines.linewidth": 3,
    }
)

In [9]:
#### misori xfab
def calcmisori(array1,array2,gid1,gid2):
    o1 = Orientation.from_euler(array1[gid1],symmetry=cubic.point_group)
    o2 = Orientation.from_euler(array2[gid2],symmetry=cubic.point_group)
    o1m=np.linalg.inv(o1.to_matrix()[0])
    o2m=np.linalg.inv(o2.to_matrix()[0])
    misori=misorien_deg(o1m,o2m)
    return misori
def misorien_deg(U1, U2):
    return np.min(Umis(U1, U2, 7), axis=0)[1]  # 7 == cubic
def translate(array,translate_x,translate_y):
    l=[]
    for i in range (array.shape[0]):
        layer=array[i,:,:]
        translated=np.roll(layer,translate_x,axis=0)
        translated=np.roll(translated,translate_y,axis=1)

        l.append(translated)
    return np.array(l)
def rotate_array_180(array):
    l=[]
    for i in range (array.shape[0]):
        layer=array[i,:,:]
        rotated=np.rot90(np.rot90(layer))
        l.append(rotated)
    return np.array(l)
def calc_com_z(grainidarray,grainid):
    com_mask=grainidarray==grainid
    com_z_array=np.sum(np.sum(com_mask,axis=1),axis=1)
    com_z=np.where(com_z_array>0)[0]
    index=[]
    weights=[]
    for i in com_z:
        index.append(i)
        weights.append(com_z_array[i])
    weights=np.array(weights)
    index=np.array(index)
    wi=np.sum((weights*index)/np.sum(weights))
    
    return wi
def calc_com_y(grainidarray,grainid):
    com_mask=grainidarray==grainid
    com_z_array=np.sum(np.sum(com_mask,axis=1),axis=0)
    com_z=np.where(com_z_array>0)[0]
    index=[]
    weights=[]
    for i in com_z:
        index.append(i)
        weights.append(com_z_array[i])
    weights=np.array(weights)
    index=np.array(index)
    wi=np.sum((weights*index)/np.sum(weights))
    
    return wi
def calc_com_z(grainidarray,grainid):
    com_mask=grainidarray==grainid
    com_z_array=np.sum(np.sum(com_mask,axis=1),axis=1)
    com_z=np.where(com_z_array>0)[0]
    index=[]
    weights=[]
    for i in com_z:
        index.append(i)
        weights.append(com_z_array[i])
    weights=np.array(weights)
    index=np.array(index)
    wi=np.sum((weights*index)/np.sum(weights))
    
    return wi
def calc_com_x(grainidarray,grainid):
    com_mask=grainidarray==grainid
    com_z_array=np.sum(np.sum(com_mask,axis=2),axis=0)
    com_z=np.where(com_z_array>0)[0]
    index=[]
    weights=[]
    for i in com_z:
        index.append(i)
        weights.append(com_z_array[i])
    weights=np.array(weights)
    index=np.array(index)
    wi=np.sum((weights*index)/np.sum(weights))
    
    return wi
def getchildgrains(parentid,postheat=True):
    if not postheat:
        return 'change function'
    mask=grainid_merged_post==parentid
    grainids=grainid_post[mask]
    return np.unique(grainids)
def calc_com_distance(grainidarray1,grainidarray2,grainid1,grainid2):
    comz1=calc_com_z(grainidarray1,grainid1)
    comz2=calc_com_z(grainidarray2,grainid2)
    z1_z2=(abs(comz1-comz2))**2
    comy1=calc_com_y(grainidarray1,grainid1)
    comy2=calc_com_y(grainidarray2,grainid2)
    y1_y2=(abs(comy1-comy2))**2
    comx1=calc_com_x(grainidarray1,grainid1)
    comx2=calc_com_x(grainidarray2,grainid2)
    x1_x2=abs(comx1-comx2)**2
    return np.sqrt(z1_z2+y1_y2+x1_x2)
def checktwins(matrix1,matrix2):
    om1=Orientation.from_matrix(matrix1,symmetry=cubic.point_group)
    om2=Orientation.from_matrix(matrix2,symmetry=cubic.point_group)
    mori=Misorientation(~om1* om2)
    mori=Misorientation(~om1* om2)
    mori.symmetry=(om1.symmetry,om1.symmetry)
    mori=mori.map_into_symmetry_reduced_zone()
    if abs(60-((mori.angle)*(180/np.pi)))<2:
        newarray= (np.round(abs(mori.axis.data[0]),1))
        if abs(newarray[1]-newarray[0])<0.21:
            if abs(newarray[2]-newarray[0])<0.21:
                return True

In [10]:
with h5py.File ('../../../september/rawdata/dream3D/step2_segment_pre_align_combined_ysz.dream3d','r') as hin:
    print (hin['DataContainers/DataContainer/CellData/'].keys())#[()]
    print (hin['DataContainers/DataContainer/Grain Data/'].keys())#[()]
    grainid1=hin['DataContainers/DataContainer/CellData/FeatureIds'][()]
    phaseid1=hin['DataContainers/DataContainer/CellData/phaseid'][()]
    nl1=hin['DataContainers/DataContainer/Grain Data/NeighborList'][()]
    #of1=hin['DataContainers/DataContainer/CellData/oldfis'][()]  
    eu1=hin['DataContainers/DataContainer/Grain Data/EulerAngles'][()]
    ml1=hin['DataContainers/DataContainer/Grain Data/MisorientationList'][()]  
    nn1=hin['DataContainers/DataContainer/Grain Data/NumNeighbors'][()]   
    nn1=nn1[:,0]
    phaseid1=phaseid1[:,:,:,0]
    #of1=of1[:,:,:,0]
    pic1=hin['DataContainers/DataContainer/CellData/phaseid_c'][()]
    pic1=pic1[:,:,:,0]
    grainid1=grainid1[:,:,:,0]


<KeysViewHDF5 ['Allvolume', 'BoundaryCells', 'BoundaryCells2', 'EulerAngles', 'FeatureIds', 'GBManhattanDistances', 'IPFColor', 'IPFColor2', 'KernelAverageMisorientations', 'Mask', 'ParentIds', 'allvolume', 'phaseid', 'phaseid_c', 'quats']>
<KeysViewHDF5 ['Active', 'AvgMisorientations', 'AvgQuats', 'EquivalentDiameters', 'EquivalentDiameters2', 'EulerAngles', 'MisorientationList', 'MisorientationList_NumNeighbors', 'NeighborList', 'NumElements', 'NumElements2', 'NumNeighbors', 'NumNeighbors2', 'ParentIds', 'Phases', 'SharedSurfaceAreaList', 'SurfaceFeatures', 'SurfaceFeatures2', 'Volumes', 'Volumes2']>


In [12]:
with h5py.File ('../../../september/rawdata/dream3D/step2_segment_post_align_combined_ysz.dream3d','r') as hin:
    print (hin['DataContainers/DataContainer/CellData/'].keys())#[()]
    print (hin['DataContainers/DataContainer/Grain Data/'].keys())#[()]
    
    grainid2=hin['DataContainers/DataContainer/CellData/FeatureIds'][()]
    phaseid2=hin['DataContainers/DataContainer/CellData/phaseid'][()]
    nl2=hin['DataContainers/DataContainer/Grain Data/NeighborList'][()]  
    ml2=hin['DataContainers/DataContainer/Grain Data/MisorientationList'][()]  
    nn2=hin['DataContainers/DataContainer/Grain Data/NumNeighbors'][()]
    eu2=hin['DataContainers/DataContainer/Grain Data/EulerAngles'][()]
    nn2=nn2[:,0]
    phaseid2=phaseid2[:,:,:,0]
    #of2=hin['DataContainers/DataContainer/CellData/oldfis'][()]  
    pic2=hin['DataContainers/DataContainer/CellData/phaseid_c'][()]   
    pic2=pic2[:,:,:,0]
    #of2=of2[:,:,:,0]
    grainid2=grainid2[:,:,:,0]


<KeysViewHDF5 ['Allvolume', 'BoundaryCells', 'BoundaryCells2', 'EulerAngles', 'FeatureIds', 'GBManhattanDistances', 'IPFColor', 'IPFColor2', 'KernelAverageMisorientations', 'Mask', 'ParentIds', 'allvolume', 'phaseid', 'phaseid_c', 'quats']>
<KeysViewHDF5 ['Active', 'AvgMisorientations', 'AvgQuats', 'EquivalentDiameters', 'EquivalentDiameters2', 'EulerAngles', 'MisorientationList', 'MisorientationList_NumNeighbors', 'NeighborList', 'NumElements', 'NumElements2', 'NumNeighbors', 'NumNeighbors2', 'ParentIds', 'Phases', 'SharedSurfaceAreaList', 'SurfaceFeatures', 'SurfaceFeatures2', 'Volumes', 'Volumes2']>


In [13]:
listmisori_post=[]
c=0
for grain in np.unique(grainid2): 
    numn=nn2[grain]
    start=c
    end=c+numn
    listn=ml2[start:end]
    c+=numn
    listmisori_post.append(list(listn))

In [14]:
### get list of neighbors to each grain id
listnb_post=[]
c=0
for grain in np.unique(grainid2): 
    numn=nn2[grain]
    start=c
    end=c+numn
    listn=nl2[start:end]
    c+=numn
    listnb_post.append(list(listn))


In [17]:
listnb_post[246]

[30, 31, 85, 122, 126, 176, 180, 193, 241, 291, 320, 364, 401]

In [6]:
cubic = Phase(point_group="m-3m", structure=Structure())
o1 = Orientation.from_euler(eu1,symmetry=cubic.point_group)
o2 = Orientation.from_euler(eu2,symmetry=cubic.point_group)

o1m = np.linalg.inv(o1.to_matrix())
o2m = np.linalg.inv(o2.to_matrix())


In [7]:
grainid_pre=grainid1
grainid_post=grainid2


In [146]:
### calculate individual misoris
misori=misorien_deg(o1m[466],o2m[119])
print (calc_com_distance(grainid_pre,grainid_post,228,322))
print (misori)

80.77698050134131
60.61728153242721


In [173]:
# Create a dict of pairs of post data tro pre data
pairs={}
for i in range(1, o2.shape[0]):
    pairs[i]=[]
    for j in range(1, o1.shape[0] - 1):
        misori=misorien_deg(o2m[i],o1m[j])
        
        #com_dist=calc_com_distance(grainid_pre,grainid_post,j,i)
        if (misori<4):
            pairs[i].append(j)
                
        #if (misori<2) or ((60-misori<2)&(misori<60)):
    print (i,end='')

1234567891011121314151617181920212223242526272829303132333435363738394041424344454647484950515253545556575859606162636465666768697071727374757677787980818283848586878889909192939495969798991001011021031041051061071081091101111121131141151161171181191201211221231241251261271281291301311321331341351361371381391401411421431441451461471481491501511521531541551561571581591601611621631641651661671681691701711721731741751761771781791801811821831841851861871881891901911921931941951961971981992002012022032042052062072082092102112122132142152162172182192202212222232242252262272282292302312322332342352362372382392402412422432442452462472482492502512522532542552562572582592602612622632642652662672682692702712722732742752762772782792802812822832842852862872882892902912922932942952962972982993003013023033043053063073083093103113123133143153163173183193203213223233243253263273283293303313323333343353363373383393403413423433443453463473483493503513523533543553563573583593603613623633643653663673683693

In [174]:
#### com distance filter for pairs
for i in pairs:
    if len(pairs[i])==0:
        continue
    l=[]
    trash=[]
    for j in pairs[i]:
        dist=calc_com_distance(grainid_pre,grainid_post,j,i)
        if dist>7:
            trash.append(j)
    for t in trash:
        pairs[i].remove(t)



In [176]:
#### get a list of grains that have been already chosen.
preheat_chosen=[]
for i in pairs:
    if len(pairs[i])==0:
        continue
    preheat_chosen.append(pairs[i][0])

In [177]:
### sort out empty grains and see if they have any twins:
for i in pairs:
    
    if len(pairs[i])==0:
        for j in range(1, o1.shape[0] - 1):
            misori=misorien_deg(o2m[i],o1m[j])
            if (abs(misori-60)<2):
                com_dist=calc_com_distance(grainid_pre,grainid_post,j,i)
                if (com_dist<10):
                    if checktwins(o2m[i],o1m[j]):
                        pairs[i].append(j)
                        print (i)
    

6
6
15
15
34
34
81
97
97
111
121
121
157
178
220
256
261
264
287
289
289
305
305
305
305
310
316
316
321
347
357
357
357
373
378
384
423
423
452
466
496
524
533
533
586
598


In [178]:
#### get a list of grains that have been already chosen.
preheat_chosen=[]
for i in pairs:
    if len(pairs[i])==0:
        continue
    preheat_chosen.append(pairs[i][0])

In [180]:
for i in pairs:
    if len(pairs[i])<1:
        vol=grainvolume(grainid_post,i)
        if vol>100:
            print (i,vol)


581 140
601 121


In [181]:
c=[]
for i in pairs:
    if len(pairs[i])==0:
        c.append(i)
print (c)

[2, 30, 93, 98, 133, 149, 183, 187, 191, 207, 221, 230, 236, 294, 303, 309, 333, 335, 353, 390, 395, 413, 422, 455, 465, 498, 499, 514, 516, 540, 544, 547, 557, 578, 581, 596, 601, 609]


In [93]:
#### get a list of grains that have been already chosen.
preheat_chosen=[]
for i in pairs:
    if len(pairs[i])==0:
        continue
    preheat_chosen.append(pairs[i][0])

In [49]:
#### recheck misori and com of every pair.

for i in pairs:
    if len(pairs[i])!=1:
        
        continue
    for j in pairs[i]:
        misori=misorien_deg(o2m[i],o1m[j])
        dist=calc_com_distance(grainid_pre,grainid_post,j,i)
        if (58>misori>2)| (dist>10):
            print (i,dist,misori,j)

85 3.1889582759601063 34.83503564950723 497
86 1.4620006548301543 9.576588625051915 236
113 5.693478587052608 50.38067508747104 458
128 16.980944050305578 27.951308474993272 134
144 24.878448884102337 44.318154820960864 70
152 2.092988768821929 43.70923622416202 390
156 22.311808212565058 50.767808978577015 347
183 4.689534441004482 7.171245063910567 364
194 21.27481799582196 25.261126874538263 411
205 7.8925650439019 52.35486555352624 435
210 52.38649620573762 0.12177848008262493 493
211 4.070333187314276 3.812142554066431 60
217 11.44081499058353 40.01184334659776 264
226 7.0303899763778785 45.582023623940735 473
281 1.9439935812651035 20.859833935141733 134
298 5.473519041675879 40.8164158281066 31
302 4.0975784158472015 51.44868857205489 31
305 3.247195449154524 39.197294166518766 469
357 5.699065652509442 57.5522602497338 146
363 9.549041023679335 7.408455751357415 536
385 6.769305255729544 43.07957918708385 91
387 5.035120967687516 17.03913787752986 191
392 5.7492473996954425 55.

In [182]:
### saved ni grain tracking as  a pickle file . This is for segmented volume version2
import pickle
with open("grain_tracking_v5_merged_phases_align_posttopre.pickle", "wb") as f:
    pickle.dump(pairs, f)


In [183]:
### saved ni grain tracking as  a pickle file . This is for segmented volume version2
import pickle
with open("grain_tracking_v5_merged_phases_align_pretopost.pickle", "wb") as f:
    pickle.dump(preheat_chosen, f)
